In [1]:
import gradio as gr
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
from PIL import Image

# Model and adapter paths
adapter_path = "/teamspace/studios/this_studio/qwen2-7b-instruct-freshness_detecter_v1/checkpoint-155"

# Set up Qwen model
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="auto",
    cache_dir="/teamspace/studios/this_studio/qwen2-7b-instruct-freshness_detecter_v1"
)

# Processor setup
processor = AutoProcessor.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", 
    cache_dir="/teamspace/studios/this_studio/qwen2-7b-instruct-freshness_detecter_v1", 
    max_pixels=720 * 28 * 28
)

# Load adapter
model.load_adapter(adapter_path)

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
from PIL import Image
import numpy as np
import gradio as gr

# Function to process the image and assess freshness
def process_freshness(image):
    # Convert numpy array to PIL Image
    pil_image = Image.fromarray(np.uint8(image))

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image", 
                    "image": pil_image  # Use the processed PIL image
                },
                {
                    "type": "text", 
                    "text": "Identify the fruit or vegetable in the image and strictly categorize its freshness as one of the following: 'extreme freshness,' 'mild freshness,' or 'worse.' The assessment should be based on observable factors such as color, texture, and signs of spoilage. Make sure to return only one of these three categories in your response, along with a brief explanation of why this category was chosen."
                    # "text": "Identify the fruit or vegetable in the image and strictly assess its freshness as 'extreme freshness,' 'mild freshness,' or 'worse' based on color, texture, and signs of spoilage. Provide a brief explanation for your assessment."
                }   
            ]
        }
    ]
    # "Identify the fruit or vegetable in the image and strictly categorize its freshness as one of the following: 'extreme freshness,' 'mild freshness,' or 'worse.' The assessment should be based on observable factors such as color, texture, and signs of spoilage. Make sure to return only one of these three categories in your response, along with a brief explanation of why this category was chosen."

    
    # Prepare input for inference
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    ).to("cuda")

    # Generate output from the Qwen model
    generated_ids = model.generate(**inputs, max_new_tokens=64)
    generated_ids_trimmed = [out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
    output_text = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    
    return output_text[0]  # Return the output as a string

# Gradio UI
def run_gradio_interface(image):
    output_text = process_freshness(image)
    return output_text

# Define Gradio interface with image input and text output
with gr.Blocks() as demo:
    
    # Add a title and description
    gr.Markdown("## Freshness Detection with Qwen Model")
    gr.Markdown(
        """
        **Upload an image of a fruit or vegetable**, and the Qwen model will identify the item and assess its freshness.
        The assessment will include categories such as 'extreme freshness,' 'mild freshness,' or 'worse,' with a brief explanation.
        """
    )
    
    # Define the layout for the interface
    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="numpy", label="Upload Image")  # Image upload box
            submit_button = gr.Button("Run Freshness Detection")  # Submit button
        
        with gr.Column():
            output_box = gr.Textbox(label="Freshness Detection Output", lines=10, max_lines=20)  # Text output

    # Add interaction to trigger inference
    submit_button.click(fn=run_gradio_interface, inputs=image_input, outputs=output_box)

# Launch the app
demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7865
* Running on public URL: https://5b5bebf247b6e9bf03.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [2]:
from PIL import Image
import numpy as np
import gradio as gr

# Shelf life mapping based on fruit type and freshness
def get_shelf_life(fruit_type, freshness):
    shelf_life_map = {
    'apple': {
        'extreme freshness': 'around 7 days',
        'mild freshness': 'around 3-5 days',
        'worse': 'less than 2 days'
    },
    'banana': {
        'extreme freshness': 'around 4 days',
        'mild freshness': 'around 2-3 days',
        'worse': 'less than 2 days'
    },
    'potato': {
        'extreme freshness': 'around 45-60 days',
        'mild freshness': 'around 10-12 days',
        'worse': 'less than 3 days'
    },
    'orange': {
        'extreme freshness': 'around 7 days',
        'mild freshness': 'around 3-5 days',
        'worse': 'less than 2 days'
    },
    'okra': {
        'extreme freshness': 'around 7-10 days',
        'mild freshness': 'around 4-6 days',
        'worse': 'less than 2 days'
    },
    'tomato': {
        'extreme freshness': 'around 4-6 days',
        'mild freshness': 'around 2-3 days',
        'worse': 'less than 2 days'
    },
    'cucumber': {
        'extreme freshness': 'around 7 days',
        'mild freshness': 'around 3-5 days',
        'worse': 'less than 2 days'
    },
    # Add more fruits or vegetables here
}

    fruit_type = fruit_type.lower()
    freshness = freshness.lower()

    if fruit_type in shelf_life_map and freshness in shelf_life_map[fruit_type]:
        return shelf_life_map[fruit_type][freshness]
    else:
        return "Unknown shelf life"

# Function to process the image and assess freshness with shelf life
def process_freshness(image):
    # Convert numpy array to PIL Image
    pil_image = Image.fromarray(np.uint8(image))

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image", 
                    "image": pil_image  # Use the processed PIL image
                },
                {
                    "type": "text", 
                    "text": "Identify the fruit or vegetable in the image and strictly categorize its freshness as one of the following: 'extreme freshness,' 'mild freshness,' or 'worse.' The assessment should be based on observable factors such as color, texture, and signs of spoilage. Make sure to return only one of these three categories in your response, along with a brief explanation of why this category was chosen."
                }   
            ]
        }
    ]
    
    # Prepare input for inference
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    ).to("cuda")

    # Generate output from the Qwen model
    generated_ids = model.generate(**inputs, max_new_tokens=64)
    generated_ids_trimmed = [out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
    output_text = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

    # Post-process the output to extract fruit type and freshness
    fruit_type = "unknown"
    freshness = "unknown"

    if "apple" in output_text.lower():
        fruit_type = "apple"
    elif "banana" in output_text.lower():
        fruit_type = "banana"
    elif "potato" in output_text.lower():
        fruit_type = "potato"
    elif "orange" in output_text.lower():
        fruit_type = "orange"
    elif "okra" in output_text.lower():
        fruit_type = "okra"
    elif "tomato" in output_text.lower():
        fruit_type = "tomato"
    elif "cucumber" in output_text.lower():
        fruit_type = "cucumber"
    # Add detection logic for other fruits/vegetables here as needed

    if "extreme freshness" in output_text.lower():
        freshness = "extreme freshness"
    elif "mild freshness" in output_text.lower():
        freshness = "mild freshness"
    elif "moderate freshness" in output_text.lower():
        freshness = "mild freshness"
    elif "worse" in output_text.lower():
        freshness = "worse"

    # Get estimated shelf life based on fruit type and freshness
    shelf_life = get_shelf_life(fruit_type, freshness)

    # Append shelf life information to the output
    output_text += f"\n\nEstimated Shelf Life for {fruit_type.capitalize()} with {freshness.capitalize()}: {shelf_life}"

    return output_text

# Gradio UI
def run_gradio_interface(image):
    output_text = process_freshness(image)
    return output_text

# Define Gradio interface with image input and text output
with gr.Blocks() as demo:
    
    # Add a title and description
    gr.Markdown("## Freshness Detection with Shelf Life Estimation")
    gr.Markdown(
        """
        **Upload an image of a fruit or vegetable**, and the Qwen model will identify the item and assess its freshness.
        Based on the freshness level, the app will also estimate the shelf life of the fruit or vegetable.
        """
    )
    
    # Define the layout for the interface
    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="numpy", label="Upload Image")  # Image upload box
            submit_button = gr.Button("Run Freshness Detection")  # Submit button
        
        with gr.Column():
            output_box = gr.Textbox(label="Freshness Detection Output", lines=10, max_lines=20)  # Text output

    # Add interaction to trigger inference
    submit_button.click(fn=run_gradio_interface, inputs=image_input, outputs=output_box)

# Launch the app
demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://8d78f2129fa656a4f4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/uvicorn/protocols/http/h11_impl.py", line 406, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/uvicorn/middleware/proxy_headers.py", line 70, in __call__
    return await self.app(scope, receive, send)
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/starlette/applications.py", line 113, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/starlette/middleware/errors.py", line 187, in __call__
    raise exc
  File "/home/zeus/miniconda3/envs/cloudspace/lib/pyt